In [0]:
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

from pyspark.mllib.classification import SVMModel, SVMWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors as MLLibVectors
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import * 
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder


from nltk.stem.porter import *
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

import pandas as pd
import string 
import re 

In [0]:
review = spark.read.load("s3://yelp-work-repo-backup-s26-urc/yelp_academic_dataset_review.json", format="json")


In [0]:
review.cache() 

Out[148]: DataFrame[business_id: string, cool: bigint, date: string, funny: bigint, review_id: string, stars: double, text: string, useful: bigint, user_id: string]

In [0]:
# remove punctuation
def remove_punct(text):
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text) 
    return nopunct
    
# binarize rating
def convert_rating(rating):
    rating = int(rating)
    if rating >=4: return 1
    else: return 0

# udf
punct_remover = udf(lambda x: remove_punct(x))
rating_convert = udf(lambda x: convert_rating(x))

# apply to review raw data
review_df = review.select('review_id', punct_remover('text'), rating_convert('stars'))

review_df = review_df.withColumnRenamed('<lambda>(text)', 'text')\
                     .withColumn('label', review_df["<lambda>(stars)"].cast(IntegerType()))\
                     .drop('<lambda>(stars)')\
                     .limit(1000000)

In [0]:
# tokenize
tok = Tokenizer(inputCol="text", outputCol="words")
review_tokenized = tok.transform(review_df)

# remove stop words
stopword_rm = StopWordsRemover(inputCol='words', outputCol='words_nsw')
review_tokenized = stopword_rm.transform(review_tokenized)

In [0]:
review_tokenized.show()

+--------------------+--------------------+-----+--------------------+--------------------+
 review_id| text|label| words| words_nsw|
+--------------------+--------------------+-----+--------------------+--------------------+
lWC-xP3rd6obsecCY...|Apparently Prides...| 1|[apparently, prid...|[apparently, prid...|
8bFej1QE5LXp4O05q...|This store is pre...| 1|[this, store, is,...|[store, pretty, g...|
NDhkzczKjLshODbqD...|I called WVM on t...| 1|[i, called, wvm, ...|[called, wvm, rec...|
T5fAqjjFooT4V0OeZ...|I ve stayed at ma...| 0|[i, ve, stayed, a...|[ve, stayed, many...|
sjm_uUcQVxab_EeLC...|The food is alway...| 1|[the, food, is, a...|[food, always, gr...|
J4a2TuhDasjn2k3wW...|This place used t...| 0|[this, place, use...|[place, used, coo...|
28gGfkLs3igtjVy61...|The setting is pe...| 0|[the, setting, is...|[setting, perfect...|
9vqwvFCBG3FBiHGmO...|Probably one of t...| 1|[probably, one, o...|[probably, one, b...|
2l_TDrQ7p-5tANOyi...|I am definitely a...| 1|[i, am, definitel...|[definitely, fan,...|
KKVFopqzcVfcubIBx...|I work in the Pru...| 1|[i, work, in, the...|[work, pru, affor...|
FdoBFTjXXMn4hVnJ5...|They NEVER seem t...| 0|[they, never, see...|[never, seem, get...|
Qv_lQb1Yu9DGsUO3r...|Disappointing Bo...| 0|[disappointing, ,...|[disappointing, ,...|
Gi5LSRmTXoL9Bp4jN...|I have been here ...| 1|[i, have, been, h...|[twice, really, g...|
m1GlqFGIN5eayrbb2...|I wish I could gi...| 0|[i, wish, i, coul...|[wish, give, zero...|
j_eh6Iw7qseluR-7K...|This is a five st...| 1|[this, is, a, fiv...|[five, star, rest...|
egHMQBhFeaLhVyC6w...|Quickly stopped i...| 1|[quickly, stopped...|[quickly, stopped...|
LBJJWJ0uNlIybMfPn...|I loved everythin...| 1|[i, loved, everyt...|[loved, everythin...|
ucFOnqgaV40oQ2YNy...|Great coffee and ...| 0|[great, coffee, a...|[great, coffee, p...|
btNWW2kdJYfwpTDyz...|Nothing special b...| 1|[nothing, special...|[nothing, special...|
3c4LFWiVUHPDQzAd8...|Definitely star...| 1|[definitely, , , ...|[definitely, , , ...|
+--------------------+--------------------+-----+--------------------+--------------------+
only showing top 20 rows

In [0]:
# count vectorizer
cv = CountVectorizer(inputCol='words_nsw', outputCol='tf')
cvModel = cv.fit(review_tokenized)
count_vectorized = cvModel.transform(review_tokenized)

In [0]:
from pyspark.ml.feature import IDF
idf = IDF().setInputCol('tf').setOutputCol('tfidf')
tfidfModel = idf.fit(count_vectorized)
tfidf_df = tfidfModel.transform(count_vectorized)

In [0]:
tfidf_df.select('tf').count()

Out[157]: 1000000

In [0]:
# split into training and testing set
splits = tfidf_df.select(['tfidf', 'label']).randomSplit([0.8,0.2],seed=100)
train = splits[0].cache()
test = splits[1].cache()

In [0]:
from pyspark.mllib.classification import SVMModel, SVMWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors as MLLibVector

In [0]:
# convert to LabeledPoint vectors
train_lb = train.rdd.map(lambda row: LabeledPoint(row[1], MLLibVectors.fromML(row[0])))
test_lb = test.rdd.map(lambda row: LabeledPoint(row[1], MLLibVectors.fromML(row[0])))

In [0]:
# SVM model
numIterations = 50
regParam = 0.3
svm = SVMWithSGD.train(train_lb, numIterations, regParam=regParam)

In [0]:
# predict
test_lb = test.rdd.map(lambda row: LabeledPoint(row[1], MLLibVectors.fromML(row[0])))
scoreAndLabels_test = test_lb.map(lambda x: (float(svm.predict(x.features)), x.label))
score_label_test = spark.createDataFrame(scoreAndLabels_test, ["prediction", "label"])

In [0]:
# F1 score
f1_eval = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
svm_f1 = f1_eval.evaluate(score_label_test)
print("F1 score: %.4f" % svm_f1)

F1 score: 0.8856

In [0]:
vocabulary = cvModel.vocabulary
weights = svm.weights.toArray()
svm_coeffs_df = pd.DataFrame({'word': vocabulary, 'weight': weights})

In [0]:
print(svm_coeffs_df.sort_values('weight'))

word weight
205 ok -0.104189
407 worst -0.100312
380 rude -0.087662
449 terrible -0.081273
97 bad -0.079594
.. ... ...
24 love 0.112983
21 best 0.128759
46 amazing 0.136044
37 delicious 0.138628
4 great 0.204593

[225824 rows x 2 columns]